Import packages and device selection

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
devide = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
print(f'Using device: {devide}')